In [ ]:
#r "nuget: Microsoft.SemanticKernel.Agents.Abstractions, 1.14.1-alpha"
#r "nuget: Microsoft.SemanticKernel.Agents.Core, 1.14.1-alpha"
#r "nuget: Microsoft.SemanticKernel.Agents.OpenAI, 1.14.1-alpha"
#r "nuget: Microsoft.SemanticKernel.Connectors.OpenAI, 1.14.1-alpha"
#r "nuget: Newtonsoft.Json, 13.0.1" 


In [ ]:
using System.ComponentModel;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.Agents.OpenAI;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Agents.Chat;
using System.IO;
using Newtonsoft.Json.Linq;


In [ ]:
var config = JObject.Parse(File.ReadAllText("config.json"));
var AOAI_apikey = config["AOAI_apikey"].ToString();
var AOAI_endpoint = config["AOAI_endpoint"].ToString();
var AOAI_deploymentname = config["AOAI_deploymentname"].ToString();


In [ ]:
Kernel kernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(
                deploymentName: AOAI_deploymentname,
                apiKey: AOAI_apikey,
                endpoint: AOAI_endpoint)
            .Build();


In [ ]:
// Lets given the Persona to Agents

string ProgramManager = """
    You are a program manager which will take the requirement and create a plan for creating app. Program Manager understands the 
    user requirements and form the detail documents with requirements and costing.
""";

string SoftwareEngineer = """
   You are Software Engieer, and your goal is create web app using HTML and JavaScript by taking into consideration all
   the requirements given by Program Manager. 
""";

string ProjectManager = """
    You are manager which will review software engineer code, and make sure all client requirements are completed. 
    You are the guardian of quality, ensuring the final product meets all specifications and receives the green light for release.
    Ask to the Brand Analyst to ensure that the Bankinter brand guidelines are followed.
     Once all client requirements are completed, you can approve the request by just responding "approve"
""";

In [ ]:
#pragma warning disable SKEXP0110, SKEXP0001 // Rethrow to preserve stack details

ChatCompletionAgent ProgramManagerAgent =
            new()
            {
                Instructions = ProgramManager,
                Name = "ProgramManagerAgent",
                Kernel = kernel
            };

ChatCompletionAgent SoftwareEngineerAgent =
            new()
            {
                Instructions = SoftwareEngineer,
                Name = "SoftwareEngineerAgent",
                Kernel = kernel
            };

ChatCompletionAgent ProjectManagerAgent =
            new()
            {
                Instructions = ProjectManager,
                Name = "ProjectManagerAgent",
                Kernel = kernel
            };





In [ ]:
string BrandAnalyst = """
    You are a Brand Analyst for Bankinter. Your goal is to ensure that all software developed adheres to the Bankinter brand guidelines, 
    specifically using the Bankinter company colors. The primary color is orange (#FF4F00), and the secondary color is white (#FFFFFF). 
    Ensure that these colors are consistently applied throughout any app.
""";

#pragma warning disable SKEXP0110, SKEXP0001 // Rethrow to preserve stack details
ChatCompletionAgent BrandAnalystAgent =
            new()
            {
                Instructions = BrandAnalyst,
                Name = "BrandAnalystAgent",
                Kernel = kernel
            };

In [ ]:
    #pragma warning disable SKEXP0110, SKEXP0001 // Rethrow to preserve stack details
    
    using System.Threading;
    
    private sealed class ApprovalTerminationStrategy : TerminationStrategy
    {
        // Terminate when the final message contains the term "approve"
        protected override Task<bool> ShouldAgentTerminateAsync(Agent agent, IReadOnlyList<ChatMessageContent> history, CancellationToken cancellationToken)
            => Task.FromResult(history[history.Count - 1].Content?.Contains("approve", StringComparison.OrdinalIgnoreCase) ?? false);
    }

In [ ]:
#pragma warning disable SKEXP0110, SKEXP0001 

        AgentGroupChat chat =
            new(ProgramManagerAgent, SoftwareEngineerAgent, ProjectManagerAgent, BrandAnalystAgent)
            {
                ExecutionSettings =
                    new()
                    {
                        TerminationStrategy =
                            new ApprovalTerminationStrategy()
                            {
                                Agents = [ProjectManagerAgent],
                                MaximumIterations = 10,
                            }
                    }
            };


In [ ]:
        #pragma warning disable SKEXP0110, SKEXP0001 
        // Invoke chat and display messages.
        string input = """
        
        I want to develop app which will provide me calculator. Keep it very simple. And get final approval from manager.
        """;

        chat.AddChatMessage(new ChatMessageContent(AuthorRole.User, input));
        Console.WriteLine($"# {AuthorRole.User}: '{input}'");

        await foreach (var content in chat.InvokeAsync())
        {
            Console.WriteLine($"# {content.Role} - {content.AuthorName ?? "*"}: '{content.Content}'");
        }